In [1]:
import pandas as pd
from fetch import *
from network_analysis import create_corpus, LDA_topics
from collections import defaultdict

Using gpu device 0: GeForce GT 640 (CNMeM is disabled, cuDNN not available)


In [2]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import pyLDAvis.gensim as gensimvis
import pyLDAvis

/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/usr/local/lib/python3.5/dist-packages/funcy/decorators.py:56: DeprecationW

Primeiro definimos uma função para extrair as comunidades dos artigos gerados pela Sabrina

In [3]:
def monta_comunidades(fname):
    communities = defaultdict(lambda:[])
    with open(fname) as f:
        while True:
            pmids = f.readline().strip().split(' ')
            commies = [c for c in f.readline().strip().split(' ') if c !='']
            if commies == []:
                break
#             print(commies)
            for p,c in zip(pmids,commies):
                communities[int(c)].append(p)
    return communities

In [4]:
comz = monta_comunidades('zika_communities.txt')
# print(comz)
comm = monta_comunidades('mers_communities.txt')
# print(comm)

## Criando os Corpora e seus vocabulários

### Zika

In [6]:
corpora_z = {}
for c, pmids in comz.items():
    corpus, dic = create_corpus('zika', filter={"MedlineCitation.PMID":{"$in":pmids}})
    if sum([len(l) for l in corpus]) == 0:
        continue #ignore when all documents are empty
    corpora_z[c] = corpus, dic
# print(corpora_z)
                                        

### MERS

In [7]:
corpora_m = {}
for c, pmids in comm.items():
    corpus, dic = create_corpus('mers', filter={"MedlineCitation.PMID":{"$in":pmids}})
    if sum([len(l) for l in corpus]) == 0:
        continue #ignore when all documents are empty
    corpora_m[c] = corpus, dic

# print(corpora_m)

### Full Corpus

In [8]:
corpora_fz = {}
corpus_fz, dic_fz = create_corpus('zika')
lda_full_z = LDA_topics(corpus_fz,dic_fz,15)

In [9]:
corpora_fm = {}
corpus_fm, dic_fm = create_corpus('mers')
lda_full_m = LDA_topics(corpus_fm,dic_fm,15)

In [10]:
@interact(c=corpora_z)
def gera_visualização(c):
    vis_data = gensimvis.prepare(lda_full_z, corpus_fz, dic_fz)
    pyLDAvis.display(vis_data)

TypeError: cannot sort an Index object in-place, use sort_values instead